This is to help work through some plex issues.

In [ ]:
#!pwsh

$cluster = "admin@equestria"
$namespace = "equestria"
$pvcName = "plex"
$podName = "plex-debug"

kubectl config use-context $cluster

In [ ]:
#!pwsh

$debugPodYaml = @"
apiVersion: v1
kind: Pod
metadata:
  name: $podName
  namespace: $namespace
  labels:
    app: plex-debug
spec:
  restartPolicy: Never
  securityContext:
    runAsUser: 568
    runAsGroup: 568
    fsGroup: 568
    fsGroupChangePolicy: OnRootMismatch
    supplementalGroups: [44, 109, 303, 568, 10000]
  containers:
  - name: toolbox
    image: 1.25.1-bookworm@sha256:c4bc0741e3c79c0e2d47ca2505a06f5f2a44682ada94e1dba251a3854e60c2bd
    command: ["/bin/bash"]
    args:
      - -c
      - |
        echo "🚀 Debug pod ready. PVC mounted at /config"
        echo "📁 Contents:"
        ls -lah /config
        echo ""
        echo "💤 Keeping pod alive. Use 'kubectl exec' to access."
        trap : TERM INT; sleep infinity & wait
    securityContext:
      allowPrivilegeEscalation: true
    volumeMounts:
    - name: plex-data
      mountPath: /config
  volumes:
  - name: plex-data
    persistentVolumeClaim:
      claimName: $pvcName
"@

Write-Host "🚀 Creating debug pod..." -ForegroundColor Cyan
$debugPodYaml | kubectl apply -f -

Write-Host "`n⏳ Waiting for pod to be ready..."
kubectl wait --for=condition=Ready pod/$podName -n $namespace --timeout=120s

Write-Host "`n✅ Debug pod ready!" -ForegroundColor Green
kubectl get pod $podName -n $namespace

Write-Host "🔧 Opening shell in debug pod..." -ForegroundColor Cyan
Write-Host "Plex config is at: /config" -ForegroundColor Yellow

In [ ]:
kubectl exec -it $podName -n $namespace -- ls -al "/config"


In [ ]:
kubectl exec -it $podName -n $namespace -- rm -rf "/config/Crash Reports"
kubectl exec -it $podName -n $namespace -- rm -rf "/config/Plug-in Support"
kubectl exec -it $podName -n $namespace -- rm -rf "/config/Setup Plex.html"

In [ ]:
#!pwsh

Write-Host "⚠️  Scaling down Plex..." -ForegroundColor Yellow
kubectl scale deployment plex -n $namespace --replicas=0

Write-Host "`nWaiting for Plex pod to terminate..."
kubectl wait --for=delete pod -n $namespace -l app.kubernetes.io/name=plex --timeout=120s

Write-Host "✅ Plex scaled down" -ForegroundColor Green


In [ ]:
#!pwsh

# $backupDate = "2025-09-30"

# kubectl exec -it $podName -n $namespace -- cp "/config/Plug-in Support/Databases/com.plexapp.plugins.library.db-$($backupDate)" "/config/Plug-in Support/Databases/com.plexapp.plugins.library.db"
# kubectl exec -it $podName -n $namespace -- cp "/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db-$($backupDate)" "/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db"
kubectl exec -it $podName -n $namespace -- rm "/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db"
kubectl exec -it $podName -n $namespace -- rm "/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db-shm"
kubectl exec -it $podName -n $namespace -- rm "/config/Plug-in Support/Databases/com.plexapp.plugins.library.blobs.db-wal"
kubectl exec -it $podName -n $namespace -- rm "/config/Plug-in Support/Databases/com.plexapp.plugins.library.db"
kubectl exec -it $podName -n $namespace -- rm "/config/Plug-in Support/Databases/com.plexapp.plugins.library.db-shm"
kubectl exec -it $podName -n $namespace -- rm "/config/Plug-in Support/Databases/com.plexapp.plugins.library.db-wal"


In [ ]:
#!pwsh

Write-Host "🔄 Scaling Plex back up..." -ForegroundColor Cyan
kubectl scale deployment plex -n $namespace --replicas=1

Write-Host "`n⏳ Waiting for Plex to be ready..."
Start-Sleep -Seconds 5
kubectl wait --for=condition=Ready pod -n $namespace -l app.kubernetes.io/name=plex --timeout=300s

Write-Host "`n✅ Plex is back up!" -ForegroundColor Green
kubectl get pods -n $namespace -l app.kubernetes.io/name=plex

In [ ]:
#!pwsh

Write-Host "🧹 Cleaning up debug pod..." -ForegroundColor Yellow
kubectl delete pod $podName -n $namespace --wait=true

Write-Host "✅ Debug pod deleted" -ForegroundColor Green